### Auren Calibration using the Tympan

Note: If using WSL to run Python in a virtual Linux environment, then you will need to attach the USB serial port to allow access from WSL.  
Follow: https://learn.microsoft.com/en-us/windows/wsl/connect-usb
- To list: usbipd list
- To bind: usbipd bind --busid <port #>
- To attach: usbipd attach --wsl --busid <port#>
- To detach: usbipd detach --busid <port #>

In [1]:
%matplotlib widget

In [2]:
# Import dependencies
import matplotlib.pyplot as plt
import os

In [3]:
# Local imports (may take 60sec to load)
from owai.core.calibrate import Calibrate
import owai.core.data_models as odm

In [4]:
cal = Calibrate(out_path='./calibrate')

In [ ]:
# Save the calibration tones -- these should be copied onto the Tympan via MTP mode

# Create the tones
cal.save_cal_tones()

In [5]:
# Connect to the tympan (if running in a virtual environment (WSL), first attach the port with usbipd attach --wsl --busid 1-8)
cal.connect()
# Turn on "MTP Mode" so we can put the wav file on to the SD card
# cal.connect_sd() #NOT ON WLS PLEASE
# Put the PLAY1.WAV, PLAY2.WAV, and PLAY3.WAV files from the
# `calibration_tones` folder into the root folder of the Typman
# Once this is done, cycle power on the Tympan


Opened:  /dev/ttyACM1


In [ ]:
cal.connect()   # Reconnect after turning back on

In [ ]:
# Place the Auren into one end of the shortest calibration tube, and the MicW into the other end
cal.collect_mic_calibration_data(odm.tube_12mm)  # Tube 1

In [ ]:
# Great, now repeat with the other 3 tubes
cal.collect_mic_calibration_data(odm.tube_14p5mm)  # Tube 2

In [ ]:
cal.collect_mic_calibration_data(odm.tube_17p5mm)  # Tube 3

In [ ]:
cal.collect_mic_calibration_data(odm.tube_20mm)  # Tube 4

In [ ]:
# Speaker calibration data
cal.collect_speaker_calibration_data(odm.simulator_1p26cc)

In [ ]:
# Save the intermin state of the entire calibration object so that we can restore from here if needed
with open("rawdata.json", "w") as fid:
    fid.write(cal.raw_data.model_dump_json())

In [ ]:
import json
with open("rawdata.json", "r") as fid:
    raw_data = json.loads(fid.read())
    raw_data["data"] = []
    raw_data["data_ref"] = []
    raw_data = odm.RawCalibrationData(**raw_data)
cal.raw_data = raw_data


In [ ]:
# As an interim part of the process we need to get the files off the Tympan
# 1. Cycle the power on the Tympan, (reattach if using WSL2), then run this cell
cal.connect()
cal.connect_sd()
# 2. Use an explorer window to drag the files `AUDIO*.WAV` into the `calibration_data` folder


In [ ]:
# 3. Cycle the power on the Tympan, then reconnnect
cal.connect()

In [ ]:
cal.raw_data.file_meta_data_ref

In [ ]:
# Load the wav recordings
cal.raw_data.load_wav(auren_files=cal.raw_data.file_meta_data, ref_files=cal.raw_data.file_meta_data_ref) 

In [ ]:
# To make sure everything worked, let's plot the data
cal.plot_calibration_data()

In [ ]:
# Now we can run the calibration routines
cal.calibrate()


In [ ]:
# MISSING METHODS!!!!
cal.plot_calibration()
cal.plot_calibration_checks()
cal.save_calibration()

In [ ]:
# Finally collect some validation data to make sure everything works as expected.
cal.connect()
cal.collect_validation_data(odm.simulator_1p26cc)
cal.collect_validation_data(odm.simulator_2cc)

In [ ]:
# As an interim part of the process we need to get the files off the Tympan
# 1. Cycle the power on the Tympan, then run this cell
cal.connect()
cal.connect_sd()
# 2. Use an explorer window to drag the files `AUDIO*.WAV` into the `validation_data` folder
# 3. Cycle the power on the Tympan

In [ ]:
# Finally, plot the validation data
cal.plot_validation()

In [ ]:
# Validate MicW calibration 
# Use 
cal.calibrateAnalogMic()